In [254]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

import re
import time

In [194]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [3]:
parent_url = 'https://auto.ru/cars/kia/all/'

In [154]:
test_data = pd.read_csv('test.csv')

In [260]:
test_data.iloc[12561]['description']

'Машина в отличном состоянии!!17.10.2020 было сделано Т.О заменено масло в коробке передач в двигателе,так же заменены все фильтра.Почти максимальная комплектация кроме люка!!! Без ключевой доступ. В ДТП не была не билась.Красился капот и передний бампер так как от времени были сколы.Капот фары туманки затянуты броне пленкой.Делали для себя супруга не ездит вся причина продажи.. Новая летняя резина. Продаю не спеша. '

In [273]:
valid_data = pd.DataFrame(columns = ['year', 'bodytype', 'kmage', 'color', 'engineDisplacement', 
                                     'enginePower', 'fuelType', 'vehicleTransmission', 'drive', 
                                     'wheel', 'state', 'owner', 'pts', 'customs', 'owningTime', 
                                     'description', 'sell_id'])

In [274]:
valid_data

,year,bodytype,kmage,color,engineDisplacement,enginePower,fuelType,vehicleTransmission,drive,wheel,state,owner,pts,customs,owningTime,description,sell_id


In [197]:
test_data.iloc[1550]

bodyType                                                внедорожник 5 дв.
brand                                                               SKODA
car_url                 https://auto.ru/cars/used/sale/skoda/kodiaq/11...
color                                                               серый
complectation_dict      {"id":"21404914","name":"Hockey Edition","avai...
description             Куплена в январе 2019 г. у официала. Производс...
engineDisplacement                                                2.0 LTR
enginePower                                                       180 N12
equipment_dict          {"cruise-control":true,"tinted-glass":true,"es...
fuelType                                                           бензин
image                   https://autoru.naydex.net/ESooa9149/468b9eoMem...
mileage                                                             34000
modelDate                                                            2016
model_info              {"code":"KODIA

In [198]:
test_data.iloc[1550]['car_url']

'https://auto.ru/cars/used/sale/skoda/kodiaq/1101318069-a7be9a63/'

In [155]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   bodyType              34686 non-null  object
 1   brand                 34686 non-null  object
 2   car_url               34686 non-null  object
 3   color                 34686 non-null  object
 4   complectation_dict    6418 non-null   object
 5   description           34686 non-null  object
 6   engineDisplacement    34686 non-null  object
 7   enginePower           34686 non-null  object
 8   equipment_dict        24690 non-null  object
 9   fuelType              34686 non-null  object
 10  image                 34686 non-null  object
 11  mileage               34686 non-null  int64 
 12  modelDate             34686 non-null  int64 
 13  model_info            34686 non-null  object
 14  model_name            34686 non-null  object
 15  name                  34686 non-null

In [6]:
test_data['bodyType'].unique()

array(['лифтбек', 'внедорожник 5 дв.', 'хэтчбек 5 дв.', 'седан',
       'компактвэн', 'универсал 5 дв.', 'пикап одинарная кабина',
       'хэтчбек 3 дв.', 'купе', 'кабриолет', 'минивэн',
       'пикап двойная кабина', 'внедорожник 3 дв.', 'родстер', 'микровэн',
       'седан 2 дв.', 'купе-хардтоп', 'фастбек', 'тарга',
       'внедорожник открытый', 'лимузин', 'пикап полуторная кабина',
       'седан-хардтоп', 'фургон'], dtype=object)

In [158]:
options = webdriver.ChromeOptions()
# options.add_argument('headless')
driver = webdriver.Chrome(options=options)

In [288]:
urls_list = []

In [283]:
driver.get('https://auto.ru/cars/kia/all/')

In [284]:
html = driver.find_elements(By.CLASS_NAME, 'Link ListingItemTitle__link')

In [285]:
rr = driver.find_elements(By.CLASS_NAME, 'ListingItemTitle__link')

In [289]:
for item in rr:
    temp = item.get_attribute('href')
    if 'used' in temp:
        urls_list.append(temp)

In [290]:
urls_list

['https://auto.ru/cars/used/sale/kia/rio/1115004557-48277309/',
 'https://auto.ru/cars/used/sale/kia/mohaves/1114604377-9616e81c/',
 'https://auto.ru/cars/used/sale/kia/rio/1115038602-7ce4341d/',
 'https://auto.ru/cars/used/sale/kia/cerato/1114739047-04c7e4fe/',
 'https://auto.ru/cars/used/sale/kia/picanto/1115071640-f5264b41/',
 'https://auto.ru/cars/used/sale/kia/ceed/1114647535-47d94b1e/',
 'https://auto.ru/cars/used/sale/kia/cerato/1115062587-7dfa0271/',
 'https://auto.ru/cars/used/sale/kia/rio/1114610706-4dec4c99/',
 'https://auto.ru/cars/used/sale/kia/rio/1115096593-7fd5da14/',
 'https://auto.ru/cars/used/sale/kia/rio/1114808411-1eff0957/',
 'https://auto.ru/cars/used/sale/kia/ceed/1115005583-ec696b37/',
 'https://auto.ru/cars/used/sale/kia/rio/1106020295-4ba62784/',
 'https://auto.ru/cars/used/sale/kia/ceed/1114620407-1c5c8b00/',
 'https://auto.ru/cars/used/sale/kia/rio/1114626178-a60ac0f8/',
 'https://auto.ru/cars/used/sale/kia/ceed/1114820059-f15e3628/',
 'https://auto.ru/cars

In [249]:
driver.get('https://auto.ru/cars/used/sale/kia/cerato/1114739047-04c7e4fe/')

In [294]:
def parse_auto_ru(url):
        
    driver.get(url)
    
    year = int(driver.find_element(By.CLASS_NAME, 'CardInfoRow_year').text.split('\n')[1])
    bodytype = driver.find_element(By.CLASS_NAME, 'CardInfoRow_bodytype').text.split('\n')[1]
    kmage = int(re.sub('\D', '', driver.find_element(By.CLASS_NAME, 'CardInfoRow_kmAge').text))
    color = driver.find_element(By.CLASS_NAME, 'CardInfoRow_color').text.split('\n')[1]
    engine = driver.find_element(By.CLASS_NAME, 'CardInfoRow_engine').text.split('/')
    engineDisplacement = float(re.findall('(\d+.\d+)', engine[0])[0])
    enginePower = int(re.findall('\d+', engine[1])[0])
    fuelType = engine[2]
    vehicleTransmission = driver.find_element(By.CLASS_NAME, 'CardInfoRow_transmission').text.split('\n')[1]
    drive = driver.find_element(By.CLASS_NAME, 'CardInfoRow_drive').text.split('\n')[1]
    wheel = driver.find_element(By.CLASS_NAME, 'CardInfoRow_wheel').text.split('\n')[1]
    state = driver.find_element(By.CLASS_NAME, 'CardInfoRow_state').text.split('\n')[1]
    owner = driver.find_element(By.CLASS_NAME, 'CardInfoRow_ownersCount').text.split('\n')[1]
    pts = driver.find_element(By.CLASS_NAME, 'CardInfoRow_pts').text.split('\n')[1]
    customs = driver.find_element(By.CLASS_NAME, 'CardInfoRow_customs').text.split('\n')[1]
    try:
        owningTime = driver.find_element(By.CLASS_NAME, 'CardInfoRow_owningTime').text.split('\n')[1]
    except NoSuchElementException:
        owningTime = None
    try:
        description = driver.find_element(By.CLASS_NAME, 'CardDescriptionHTML').text
    except NoSuchElementException:
        description = None
    sell_id = int(re.findall('\d+', driver.find_element(By.CLASS_NAME, 'CardHead__id').text)[0])
    
    data_dict = {
        'year': year, 'bodytype': bodytype, 'kmage': kmage, 'color': color, 
        'engineDisplacement': engineDisplacement, 'enginePower': enginePower, 
        'fuelType': fuelType, 'vehicleTransmission': vehicleTransmission, 'drive': drive, 
        'wheel': wheel, 'state': state, 'owner': owner, 'pts': pts, 'customs': customs, 
        'owningTime': owningTime, 'description': description, 'sell_id': sell_id
    }
    
    return data_dict

In [295]:
for url in urls_list:
    valid_data = valid_data.append(parse_auto_ru(url), ignore_index=True)

In [296]:
valid_data

,year,bodytype,kmage,color,engineDisplacement,enginePower,fuelType,vehicleTransmission,drive,wheel,state,owner,pts,customs,owningTime,description,sell_id
0,2013,седан,85355,серый,1.6,130,Бензин,автоматическая,передний,Левый,Не требует ремонта,3 или более,Дубликат,Растаможен,None,"Автомобиль был принят в зачёт на новый, прошел...",1114739047
1,2019,хэтчбек 5 дв.,61700,чёрный,1.6,123,Бензин,автоматическая,передний,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,None,None,1115004557
2,2012,внедорожник 5 дв.,130000,чёрный,3.8,274,Бензин,автоматическая,полный,Левый,Не требует ремонта,2 владельца,Оригинал,Растаможен,None,Фактически 2 владельца. Автомобиль был в одной...,1114604377
3,2018,седан,114000,чёрный,1.6,123,Бензин,автоматическая,передний,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,None,- ПТС оригинал\n- 1 собственник\n- Заводской о...,1115038602
4,2013,седан,85355,серый,1.6,130,Бензин,автоматическая,передний,Левый,Не требует ремонта,3 или более,Дубликат,Растаможен,None,"Автомобиль был принят в зачёт на новый, прошел...",1114739047
5,2014,хэтчбек 3 дв.,117025,серый,1.3,85,Бензин,автоматическая,передний,Левый,Не требует ремонта,2 владельца,Оригинал,Растаможен,None,Автомобиль приобретался новым у Официального Д...,1115071640
6,2018,хэтчбек 5 дв.,51100,серебристый,1.6,128,Бензин,автоматическая,передний,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,None,Автомобиль был куплен в 2018 году у официально...,1114647535
7,2017,седан,70000,чёрный,1.6,130,Бензин,механическая,передний,Левый,Не требует ремонта,2 владельца,Оригинал,Растаможен,None,Машина огонь. Все запчасти заменены в срок.\nН...,1115062587
8,2019,седан,44000,серый,1.6,123,Бензин,автоматическая,передний,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,None,При покупке данного автомобиля в кредит вы пол...,1114610706
9,2015,седан,207,серебристый,1.4,107,"Бензин, газобаллонное оборудование",механическая,передний,Левый,Не требует ремонта,3 или более,Оригинал,Растаможен,1 год,Автомобиль на ходу\nсостояние как на фото\nест...,1115096593


In [255]:
try:
    owningTime = driver.find_element(By.CLASS_NAME, 'CardInfoRow_owningTime')
except NoSuchElementException:
    print('no such url')

no such url


In [266]:
sell_id = driver.find_element(By.CLASS_NAME, 'CardHead__id')

In [270]:
int(re.findall('\d+', sell_id.text)[0])

1114739047

In [204]:
engine_part = engine.text.split('/')[0]

In [212]:
float(re.findall('(\d+.\d+)', engine_part)[0])

1.6

In [272]:
print(year, bodytype, kmage, color, engineDisplacement, enginePower, fuelType, vehicleTransmission, 
      drive, wheel, state, owner, pts, customs, owningTime, sell_id)

седан 2013 85355 серый 1.6 130  Бензин автоматическая передний Левый Не требует ремонта 3 или более Дубликат Растаможен None 1114739047


In [81]:
all_links = res.find_all('a', class_='Link ListingItemTitle__link')

In [82]:
for item in all_links:
    urls_list.append(item.get('href'))

In [83]:
urls_list

['https://auto.ru/cars/used/sale/audi/80/1115092543-8feec6ba/',
 'https://auto.ru/cars/used/sale/audi/a4/1114778823-f950698e/',
 'https://auto.ru/cars/used/sale/audi/a4/1106399513-17514c84/',
 'https://auto.ru/cars/used/sale/audi/a5/1115092301-5f86aa3b/',
 'https://auto.ru/cars/used/sale/audi/a6/1115093718-d5bf6834/',
 'https://auto.ru/cars/used/sale/audi/q5/1115038520-66b0c57e/',
 'https://auto.ru/cars/used/sale/audi/a6/1115038368-062d8777/',
 'https://auto.ru/cars/used/sale/audi/q5/1115031253-23a0eee3/',
 'https://auto.ru/cars/used/sale/audi/q3/1115002678-1a80794a/',
 'https://auto.ru/cars/used/sale/audi/a6/1114672329-c359b4f1/',
 'https://auto.ru/cars/used/sale/audi/a6/1105241378-5996ace8/',
 'https://auto.ru/cars/used/sale/audi/q5/1115023778-18e0ada4/',
 'https://auto.ru/cars/used/sale/audi/allroad/1115022778-7a68db68/',
 'https://auto.ru/cars/used/sale/audi/a8/1115022151-e919d8c9/',
 'https://auto.ru/cars/used/sale/audi/q7/1106592784-c73e0464/',
 'https://auto.ru/cars/used/sale/au